In [ ]:
name_list = ['Wiper Blades', 'Front Bumper', 'Front Fender', 'Bonnet', 'Front Door', 'Bootlid']

In [57]:
url  = 'https://droom.in/repair-estimate'
# url = 'https://droom.in/eco/repair-estimate'

In [58]:
import requests
import re
from bs4 import BeautifulSoup
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like
result = requests.get(url, headers = headers)
src = result.content
soup = BeautifulSoup(src, 'lxml')

In [59]:
make = soup.find(text = re.compile("Select Car to Check Estimated Repair Cost")).parent.parent.find_all('select', attrs = {'name':'make'})
option_list = make[0].find_all('option' )

In [61]:
option_list = make[0].find_all('option' )
# option_list

In [64]:
make  = make_list[2]['value']
make_id = make_list[2]['data-make-id']
make, make_id

('Audi', '801')

In [65]:
getModelList = 'https://droom.in/getRepairServiceModelList?make='+make_list[2]['value']+'&make_id='+make_list[2]['data-make-id']+'&bucket=cars'

In [127]:
sample = "apple cidar"
sample_list = sample.split()
'+'.join(sample_list)

'apple+cidar'

In [190]:
# make_list

In [3]:
import pdb
import json
from tqdm.notebook import tqdm
import requests
import re
from bs4 import BeautifulSoup

In [4]:
url  = 'https://droom.in/repair-estimate'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like
result = requests.get(url, headers = headers)
src = result.content
soup = BeautifulSoup(src, 'lxml')
make = soup.find(text = re.compile("Select Car to Check Estimated Repair Cost")).parent.parent.find_all('select', attrs = {'name':'make'})
option_list = make[0].find_all('option' )
make_list = []
for option in option_list:
    make_list.append(option.attrs)
make_list = make_list[1:]
# make_list
names_list = ['Wiper Blades', 'Front Bumper', 'Front Fender', 'Bonnet', 'Front Door', 'Bootlid', 'Front Windscreen Glass', 'Side Mirrors', 'Front Strut', 'Front Suspension Lower Arm','Suspension Overhaul']

In [10]:
make_list[53]

{'value': 'Toyota', 'data-make-id': '41'}

In [42]:
make_list[53]

{'value': 'Toyota', 'data-make-id': '41'}

In [43]:
#maruti suzuki remains, id = 38, 41, 49,52,53
for make_dict in tqdm(make_list[53:54]):
    # print(make_dict)
    make = make_dict['value'].split()
    make = '+'.join(make)
    make_id = make_dict['data-make-id']
    getModelList = 'https://droom.in/getRepairServiceModelList?make='+make+'&make_id='+make_id+'&bucket=cars'
    # print(make, make_id)
    model_call.status_code = 504
    while(model_call.status_code == 504):
        model_call = requests.get(getModelList, headers = headers, timeout=None)
    model_response = model_call.content
    model_list = json.loads(model_response.decode('utf-8'))['data']
    for model_dict in tqdm(model_list):
        model = model_dict['model_org_name'].split()
        model = '+'.join(model)
        model_id = model_dict['model_id']
        # print(make, make_id, model, model_id)
        getTrimList = "https://droom.in/getRepairServiceTrimList?make="+make+"&make_id="+make_id+"&model="+model+"&model_id="+model_id+"&bucket=cars"
        # print(getTrimList)
        model_call.status_code = 504
        while(model_call.status_code == 504):
            model_call = requests.get(getTrimList, headers = headers, timeout=None)
        trim_response = model_call.content
        trim_list = json.loads(trim_response.decode('utf-8'))['data']
        for trim_dict in tqdm(trim_list):
            if 'trim_org_name' in trim_dict.keys():
                trim = trim_dict['trim_org_name'].split()
                trim = '+'.join(trim)
                trim_id = trim_dict['trim_id']
                if trim_id != '':
                    print(make, make_id, model, model_id, trim, trim_id)
                    getTrimDetails = "https://droom.in/getRepairServiceTrimDetails?make="+make+"&make_id="+make_id+"&model="+model+"&model_id="+model_id+"&bucket=cars&trim="+trim+"&trim_id="+trim_id
                    # print(getTrimDetails)
                    model_call.status_code = 504
                    while(model_call.status_code == 504):
                        model_call = requests.get(getTrimDetails, headers = headers, timeout=None)
                        print("status code is 504")
                    service_response = model_call.content
                    service_dict = json.loads(service_response.decode('utf-8'))
                # print(service_dict)
                    if 'data' in service_dict.keys():
                        fuel_type = service_dict['data']['fuel_type']
                        body_type = service_dict['data']['body_type']
                        if fuel_type != '':
                            getServiceList =  "https://droom.in/getRepairServicesList?trim_id="+trim_id+"&body_type="+body_type+"&fuel_type="+fuel_type
                            model_call.status_code = 504
                            while( model_call.status_code == 504):
                                model_call = requests.get(getServiceList, headers = headers, timeout=None)
                            result = model_call.content
                            # print(result)
                            result_dict = json.loads(result.decode('utf-8'))
                            if result_dict['data'] != '':
                                prices_soup = BeautifulSoup(result, 'lxml')
                                body_soup = prices_soup.find_all('div', attrs = {'id':'\\"outer-body-repair-service\\"'})
                                parts = body_soup[0].find_all('input')
                                prices_list = []
                                for part, part_name in zip(parts, names_list):
                                    prices_dict = {}
                                    # part_name = part.attrs['value'].strip('\\"')
                                    prices_dict['part_name'] = part_name
                                    price_min = part.attrs['data-price-min'].strip('\\"')
                                    prices_dict['price_min'] = price_min
                                    price_max = part.attrs['data-price-max'].strip('\\"')
                                    prices_dict['price_max'] = price_max
                                    prices_list.append(prices_dict)
                                service_dict['prices_list'] = prices_list
                                # print(prices_list)
                                trim_dict['data'] = service_dict
        model_dict['trim_list'] = trim_list
    make_dict['model_list'] = model_list

Toyota 41 Alphard 1001 V6 15251
status code is 504



Toyota 41 Camry 981 2.4 208981
status code is 504
Toyota 41 Camry 981 2.4+AT 491031
status code is 504
Toyota 41 Camry 981 2.5+AT 208931
status code is 504
Toyota 41 Camry 981 ACV30R 492191
status code is 504
Toyota 41 Camry 981 HYBRID 208941
status code is 504
Toyota 41 Camry 981 HYBRID+BS6 584478
status code is 504
Toyota 41 Camry 981 V1 208961
status code is 504
Toyota 41 Camry 981 V3 208971
status code is 504
Toyota 41 Camry 981 V4+MT 208991
status code is 504
Toyota 41 Camry 981 V6+AT 209001
status code is 504
Toyota 41 Camry 981 W1 209071
status code is 504
Toyota 41 Camry 981 W2+AT 209031
status code is 504
Toyota 41 Camry 981 W3 209081
status code is 504
Toyota 41 Camry 981 W4+AT 209041
status code is 504



Toyota 41 Corolla 871 1.8E 209271
status code is 504
Toyota 41 Corolla 871 E 209231
status code is 504
Toyota 41 Corolla 871 G 209111
status code is 504
Toyota 41 Corolla 871 H1+1.8+J 209181
status code is 504
Toyota 41 Corolla 871 H2+1.8+E 209171
status code is 504
Toyota 41 Corolla 871 H3+1.8+G 209191
status code is 504
Toyota 41 Corolla 871 H4+1.8+G 209201
status code is 504
Toyota 41 Corolla 871 H5+1.8+E 209211
status code is 504
Toyota 41 Corolla 871 H7 209261
status code is 504
Toyota 41 Corolla 871 HE+1.8J 209251
status code is 504



Toyota 41 Corolla+Altis 911 1.8+G 489541
status code is 504
Toyota 41 Corolla+Altis 911 1.8+G+AT 489551
status code is 504
Toyota 41 Corolla+Altis 911 1.8+G+CNG 489561
status code is 504
Toyota 41 Corolla+Altis 911 1.8+GL 489531
status code is 504
Toyota 41 Corolla+Altis 911 1.8+GL+CNG 489571
status code is 504
Toyota 41 Corolla+Altis 911 1.8+J 489481
status code is 504
Toyota 41 Corolla+Altis 911 1.8+J+CNG 489581
status code is 504
Toyota 41 Corolla+Altis 911 1.8+J+S 489651
status code is 504
Toyota 41 Corolla+Altis 911 1.8+SPORT 489491
status code is 504
Toyota 41 Corolla+Altis 911 1.8+SPORTS 489511
status code is 504
Toyota 41 Corolla+Altis 911 1.8+V 489721
status code is 504
Toyota 41 Corolla+Altis 911 1.8+VL 489591
status code is 504
Toyota 41 Corolla+Altis 911 1.8+VL+AT 489641
status code is 504
Toyota 41 Corolla+Altis 911 1.8V 489621
status code is 504
Toyota 41 Corolla+Altis 911 AERO+LIMITED+EDITION 489661
status code is 504
Toyota 41 Corolla+Altis 911 AERO+LIMITED+EDITION+DIES

Toyota 41 Corolla+Levin 1031 FZ 451071
status code is 504



...

Toyota 41 Crown 1051 ROYAL+SALOON 15311
status code is 504



Toyota 41 Etios 901 G 482461
status code is 504
Toyota 41 Etios 901 GD 482471
status code is 504
Toyota 41 Etios 901 GD+SP 482571
status code is 504
Toyota 41 Etios 901 GX 482641
status code is 504
Toyota 41 Etios 901 J+PS 482511
status code is 504
Toyota 41 Etios 901 V 482481
status code is 504
Toyota 41 Etios 901 VD 482531
status code is 504



Toyota 41 Etios+Liva 921 G 14161
status code is 504
Toyota 41 Etios+Liva 921 G+SP 14141
status code is 504
Toyota 41 Etios+Liva 921 GD 14291
status code is 504
Toyota 41 Etios+Liva 921 GD+SP 14301
status code is 504
Toyota 41 Etios+Liva 921 GX 14351
status code is 504
Toyota 41 Etios+Liva 921 GXD 14341
status code is 504
Toyota 41 Etios+Liva 921 J 14271
status code is 504
Toyota 41 Etios+Liva 921 TRD+SPORTIVE+PETROL 14281
status code is 504
Toyota 41 Etios+Liva 921 TRD+SPORTIVO+DIESEL 14151
status code is 504
Toyota 41 Etios+Liva 921 V 14191
status code is 504
Toyota 41 Etios+Liva 921 V+DUAL+TONE 515371
status code is 504
Toyota 41 Etios+Liva 921 V+SP 14201
status code is 504
Toyota 41 Etios+Liva 921 VD 14221
status code is 504
Toyota 41 Etios+Liva 921 VD+DUAL+TONE 515391
status code is 504
Toyota 41 Etios+Liva 921 VD+SP 14211
status code is 504
Toyota 41 Etios+Liva 921 VX 14311
status code is 504
Toyota 41 Etios+Liva 921 VX+DUAL+TONE 515381
status code is 504
Toyota 41 Etios+Liva 921 

Toyota 41 Fortuner 971 2.5+4X2+AT+TRD+SPORTIVO 168731
status code is 504
Toyota 41 Fortuner 971 2.5+4X2+MT+TRD+SPORTIVO 168741
status code is 504
Toyota 41 Fortuner 971 2.7+4X2+AT 168681
status code is 504
Toyota 41 Fortuner 971 2.7+4x2+AT+BS6 584468
status code is 504
Toyota 41 Fortuner 971 2.7+4X2+MT 168661
status code is 504
Toyota 41 Fortuner 971 2.7+4x2+MT+BS6 584467
status code is 504
Toyota 41 Fortuner 971 2.8+4X2+AT 168691
status code is 504
Toyota 41 Fortuner 971 2.8+4x2+AT+BS6 584470
status code is 504
Toyota 41 Fortuner 971 2.8+4X2+MT 168671
status code is 504
Toyota 41 Fortuner 971 2.8+4x2+MT+BS6 584469
status code is 504
Toyota 41 Fortuner 971 2.8+4X4+AT 168711
status code is 504
Toyota 41 Fortuner 971 2.8+4x4+AT+BS6 584472
status code is 504
Toyota 41 Fortuner 971 2.8+4X4+MT 168701
status code is 504
Toyota 41 Fortuner 971 2.8+4x4+MT+BS6 584471
status code is 504
Toyota 41 Fortuner 971 3.0+4X2+AT 168631
status code is 504
Toyota 41 Fortuner 971 3.0+4X4+AT 168601
status co

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [40]:
make_list[52]


{'value': 'Tata', 'data-make-id': '21'}

In [39]:
for i in range(49,50):
    name = make_list[i]['value'].split()
    name = '_'.join(name)+'.json'
    print(name)
    with open(name, "w") as outfile:  
        json.dump(make_list[i], outfile) 

Renault.json


In [170]:
model_call = requests.get('https://droom.in/getRepairServiceTrimDetails?make=Ashok+Leyland&make_id=461&model=Stile&model_id=5311&bucket=cars&trim=LE&trim_id=444771', headers  = headers)
service_response = model_call.content

In [171]:
service_response

b'{"code":"success","data":{"fuel_type":"diesel","body_type":"muv","vehicle_image":"https:\\/\\/cdn1.acedms.com\\/uploads\\/category\\/ashok-leyland\\/20171108110033000000-1859412813505688321.png"}}'

In [160]:
getTrimDetails

'https://droom.in/getRepairServiceTrimDetails?make=Audi&make_id=801&model=A3&model_id=8341&bucket=cars&trim=&trim_id='

In [66]:
getModelList

'https://droom.in/getRepairServiceModelList?make=Audi&make_id=801&bucket=cars'

In [67]:
model_call = requests.get(getModelList, headers = headers)
# model_call.content

In [68]:
model_response = model_call.content
type(model_response)

bytes

In [69]:
import json

In [124]:
model_dict = json.loads(model_response.decode('utf-8')) 
model_dict

In [71]:
model_list = model_dict['data']
model_list[0]

{'model_org_name': 'A3',
 'ex_showroom_price': 4475000,
 'body_type': 'sedan',
 'model': 'a3',
 'model_id': '8341',
 'make_id': '801',
 'make': 'audi'}

In [72]:
model = model_list[0]['model_org_name']
model_id = model_list[0]['model_id']
model, model_id

('A3', '8341')

In [18]:
#print all available models
# for item in model_list:
#     print(item['model_org_name'])

In [73]:
make, make_id, model, model_id

('Audi', '801', 'A3', '8341')

In [74]:
getTrimList = "https://droom.in/getRepairServiceTrimList?make="+make+"&make_id="+make_id+"&model="+model+"&model_id="+model_id+"&bucket=cars"

In [75]:
getTrimList

'https://droom.in/getRepairServiceTrimList?make=Audi&make_id=801&model=A3&model_id=8341&bucket=cars'

In [76]:
model_call = requests.get(getTrimList, headers = headers)
trim_response = model_call.content

In [77]:
trim_dict = json.loads(trim_response.decode('utf-8')) 

In [78]:
trim_dict['data'][0]

{'trim': '1.8 tfsi',
 'model_org_name': 'A3',
 'ex_showroom_price': 2955400,
 'body_type': 'sedan',
 'model': 'a3',
 'model_id': '8341',
 'fuel_type': 'petrol',
 'make_id': '801',
 'trim_id': '376611',
 'make': 'audi',
 'trim_org_name': '1.8 TFSI'}

In [79]:
trim = trim_dict['data'][0]['trim_org_name'].split()
trim = '+'.join(trim)
trim_id = trim_dict['data'][0]['trim_id']
trim, trim_id
getTrimDetails = "https://droom.in/getRepairServiceTrimDetails?make=Audi&make_id=801&model=A3&model_id=8341&bucket=cars&trim="+trim+"&trim_id="+trim_id

('1.8+TFSI', '376611')

In [80]:
getTrimDetails = "https://droom.in/getRepairServiceTrimDetails?make=Audi&make_id=801&model=A3&model_id=8341&bucket=cars&trim="+trim+"&trim_id="+trim_id
getTrimDetails

'https://droom.in/getRepairServiceTrimDetails?make=Audi&make_id=801&model=A3&model_id=8341&bucket=cars&trim=1.8+TFSI&trim_id=376611'

In [52]:
getTrimDetails = 'https://droom.in/getRepairServiceTrimDetails?make=Audi&make_id=801&model=A3&model_id=8341&bucket=cars&trim=1.8+TFSI&trim_id=376611'

In [61]:
model_call = requests.get(getTrimDetails, headers  = headers)
service_response = model_call.content

In [62]:
service_response

b'{"code":"success","data":{"fuel_type":"petrol","body_type":"sedan","vehicle_image":"https:\\/\\/cdn1.acedms.com\\/uploads\\/category\\/audi\\/20170926092600000000-7586353322518107137.png"}}'

In [63]:
service_dict = json.loads(service_response.decode('utf-8'))
service_dict

{'code': 'success',
 'data': {'fuel_type': 'petrol',
  'body_type': 'sedan',
  'vehicle_image': 'https://cdn1.acedms.com/uploads/category/audi/20170926092600000000-7586353322518107137.png'}}

In [55]:
getServiceList = "https://droom.in/getRepairServicesList?trim_id=376611&body_type=sedan&fuel_type=petrol"

In [57]:
model_call = requests.get(getServiceList, headers  = headers)
service_response = model_call.content
# service_response

In [17]:
if 'dat' in service_dict.keys():
    print('fasf')

In [64]:
if 'data' in service_dict.keys():
    fuel_type = service_dict['data']['fuel_type']
    body_type = service_dict['data']['body_type']
    getServiceList =  "https://droom.in/getRepairServicesList?trim_id="+trim_id+"&body_type="+body_type+"&fuel_type="+fuel_type
    model_call = requests.get(getServiceList, headers = headers)
    result = model_call.content
    prices_soup = BeautifulSoup(result, 'lxml')
    body_soup = prices_soup.find_all('div', attrs = {'id':'\\"outer-body-repair-service\\"'})
    parts = body_soup[0].find_all('input')
    prices_list = []
    for part, part_name in zip(parts, names_list):
        prices_dict = {}
        # part_name = part.attrs['value'].strip('\\"')
        prices_dict['part_name'] = part_name
        price_min = part.attrs['data-price-min'].strip('\\"')
        prices_dict['price_min'] = price_min
        price_max = part.attrs['data-price-max'].strip('\\"')
        prices_dict['price_max'] = price_max
        prices_list.append(prices_dict)
    service_dict['prices_list'] = prices_list

In [65]:
service_dict

{'code': 'success',
 'data': {'fuel_type': 'petrol',
  'body_type': 'sedan',
  'vehicle_image': 'https://cdn1.acedms.com/uploads/category/audi/20170926092600000000-7586353322518107137.png'},
 'prices_list': [{'part_name': 'Wiper Blades',
   'price_min': '540',
   'price_max': '660'},
  {'part_name': 'Front Bumper', 'price_min': '2080', 'price_max': '2540'},
  {'part_name': 'Front Fender', 'price_min': '1720', 'price_max': '2100'},
  {'part_name': 'Bonnet', 'price_min': '4030', 'price_max': '4920'},
  {'part_name': 'Front Door', 'price_min': '6150', 'price_max': '7520'},
  {'part_name': 'Bootlid', 'price_min': '4690', 'price_max': '5730'},
  {'part_name': 'Front Windscreen Glass',
   'price_min': '3130',
   'price_max': '3830'},
  {'part_name': 'Side Mirrors', 'price_min': '1280', 'price_max': '1560'},
  {'part_name': 'Front Strut', 'price_min': '1600', 'price_max': '1950'},
  {'part_name': 'Front Suspension Lower Arm',
   'price_min': '1650',
   'price_max': '2020'},
  {'part_name': 'S

In [40]:
parts[4]

<input body="" data-price-max='\"7310\"' data-price-min='\"5980\"' data-service-head='\"Outer' door="" repair="" service="" type='\"checkbox\"' value='\"Front'/>

In [85]:
model_call = requests.get(getServiceList, headers = headers)
result = model_call.content

In [27]:
prices_soup = BeautifulSoup(result, 'lxml')
# prices_soup

In [28]:
body_soup = prices_soup.find_all('div', attrs = {'id':'\\"outer-body-repair-service\\"'})
# body_soup

In [29]:
parts = body_soup[0].find_all('input')
# parts

In [9]:
names_list = ['Wiper Blades', 'Front Bumper', 'Front Fender', 'Bonnet', 'Front Door', 'Bootlid', 'Front Windscreen Glass', 'Side Mirrors', 'Front Strut', 'Front Suspension Lower Arm','Suspension Overhaul']

In [116]:
part_name = parts[1].attrs['value']

In [119]:
part_name.strip('\\"')

'Front'

In [121]:
parts[0].attrs['data-price-min'].strip('\\"'), parts[0].attrs['data-price-max'].strip('\\"')

('540', '660')

In [43]:
parts[10]

<input body="" data-price-max='\"3300\"' data-price-min='\"2700\"' data-service-head='\"Outer' overhaul="" repair="" service="" type='\"checkbox\"' value='\"Suspension'/>